# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, r2_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package punkt to C:\Users\Michael
[nltk_data]     Partel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Michael
[nltk_data]     Partel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Michael
[nltk_data]     Partel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
with engine.connect() as conn:
    df = pd.read_sql_table('Figure8Messages',conn)
X = df[['message']]
X = X.message.tolist()
Y = df.drop(axis=1, columns=['id', 'message', 'original','genre'])

In [3]:
Y.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26174.000000,26174.000000,26174.00000,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000,26174.0,...,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000,26174.000000
mean,0.773516,0.170551,0.00447,0.414113,0.079468,0.050050,0.027661,0.017995,0.032819,0.0,...,0.011806,0.043975,0.278368,0.082104,0.093222,0.010774,0.093681,0.020173,0.052571,0.193436
std,0.435299,0.376123,0.06671,0.492578,0.270473,0.218052,0.164003,0.132935,0.178166,0.0,...,0.108012,0.205043,0.448204,0.274529,0.290749,0.103239,0.291390,0.140593,0.223180,0.395000
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
for col in Y.columns:
    if Y[col].max() == 0:
        Y = Y.drop(axis=1, columns=[col])

Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    clean_tokens = []
    for tok in tokens:
        if tok not in stop_words:
            clean_tok = stemmer.stem(lemmatizer.lemmatize(tok))
            clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#Split
X_train, X_test, y_train, y_test = train_test_split(X,Y, train_size=0.7, random_state=42)

#Fit
pipeline.fit(X_train, y_train)

#Predict
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
for idx, col in enumerate(y_test.columns):
    print(col)
    confusion_mat = confusion_matrix(y_test[col], y_pred[:,idx], labels=np.unique(y_pred[:,idx]))
    print('----------------------------------')
    print(classification_report(y_test[col], y_pred[:,idx]))
    print("Confusion Matrix:\n\n", confusion_mat)
    print("Accuracy: {}".format((y_test[col] == y_pred[:,idx]).mean()))
    print("\n")

related
----------------------------------
              precision    recall  f1-score   support

           0       0.72      0.42      0.53      1800
           1       0.84      0.95      0.89      5987
           2       0.77      0.15      0.25        66

    accuracy                           0.82      7853
   macro avg       0.78      0.51      0.56      7853
weighted avg       0.81      0.82      0.80      7853

Confusion Matrix:

 [[ 752 1048    0]
 [ 285 5699    3]
 [   5   51   10]]
Accuracy: 0.8227429008022412


request
----------------------------------
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      6506
           1       0.84      0.48      0.61      1347

    accuracy                           0.90      7853
   macro avg       0.87      0.73      0.78      7853
weighted avg       0.89      0.90      0.88      7853

Confusion Matrix:

 [[6382  124]
 [ 694  653]]
Accuracy: 0.895835986247294


offer
-----------------

C:\Users\Michael Partel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9844645358461734


money
----------------------------------
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7667
           1       0.60      0.02      0.03       186

    accuracy                           0.98      7853
   macro avg       0.79      0.51      0.51      7853
weighted avg       0.97      0.98      0.97      7853

Confusion Matrix:

 [[7665    2]
 [ 183    3]]
Accuracy: 0.9764421240290335


missing_people
----------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7780
           1       1.00      0.03      0.05        73

    accuracy                           0.99      7853
   macro avg       1.00      0.51      0.52      7853
weighted avg       0.99      0.99      0.99      7853

Confusion Matrix:

 [[7780    0]
 [  71    2]]
Accuracy: 0.9909588692219534


refugees
----------------------------------
              precision 

C:\Users\Michael Partel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Michael Partel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Michael Partel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Michael Partel\anac

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.steps

[('vect',
  CountVectorizer(tokenizer=<function tokenize at 0x00000229E0D14940>)),
 ('tfidf', TfidfTransformer()),
 ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))]

In [10]:
parameters = {
    'vect__max_df': [0.5, 0.75, 1],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators' : [100, 250, 500],
    'clf__estimator__min_samples_leaf': [1, 2, 4]
}    

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.best_estimator_.steps

In [ ]:
y_pred2 = cv.predict(X_test)

In [ ]:
for idx, col in enumerate(y_test.columns):
    print(col)
    confusion_mat = confusion_matrix(y_test[col], y_pred2[:,idx], labels=np.unique(y_pred2[:,idx]))
    print('----------------------------------')
    print(classification_report(y_test[col], y_pred2[:,idx]))
    print("Confusion Matrix:\n\n", confusion_mat)
    print("Accuracy: {}".format((y_test[col] == y_pred2[:,idx]).mean()))
    print("\n")

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipelineNB = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MultinomialNB()))
    ])

In [ ]:
cv.get_params().keys()

In [ ]:
parameters = {
    'vect__max_df': [0.5, 0.75, 1],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__use_idf': (True, False),
    'tfidf__min_df' : [1, 2, 5],
    'clf__alpha' : [0.0, 1.0],
} 
cvNB = GridSearchCV(pipelineNB, param_grid=parameters)

cvNB.fit(X_train, y_train)

In [ ]:
cvNB.best_estimator_.steps

In [ ]:
y_pred3 = cv.predict(X_test)

In [ ]:
for idx, col in enumerate(y_test.columns):
    print(col)
    confusion_mat = confusion_matrix(y_test[col], y_pred3[:,idx], labels=np.unique(y_pred3[:,idx]))
    print('----------------------------------')
    print(classification_report(y_test[col], y_pred3[:,idx]))
    print("Confusion Matrix:\n\n", confusion_mat)
    print("Accuracy: {}".format((y_test[col] == y_pred3[:,idx]).mean()))
    print("\n")

### 9. Export your model as a pickle file

In [ ]:
from joblib import dump, load
dump(pipeline, 'pipeline.pkl')
dump(cv.best_estimator_, 'cv_best.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.